In [1]:
from Seva import Seva
# import mysql.connector
obj = Seva()

In [7]:
#tips, prop, travel
obj.getAllBerita([], 1, 'hobi-komunitas', 'otomotif')

page  1
https://www.seva.id/otomotif/blog/category/hobi-komunitas/page/1
memasukkan berita id  871315
Insert berita  Digelar 2 Hari, BimmerJunction 2018 Lebih Seru dari Sebelumnya
masuk
memasukkan berita id  868165
Insert berita  Cerita dari Rumah Para Pecinta Mobil BMW
masuk
memasukkan berita id  868012
Insert berita  Cara Indonesia Peugeot 306 Community Berbagi Kebahagiaan
masuk
memasukkan berita id  867940
Insert berita  Mobil Super Mewah Pemain Sepakbola Dunia, Siapa Paling Wah?
masuk
memasukkan berita id  865499
Insert berita  Hauwke Auto Gallery Membawa Aura Retro dengan Mobil Klasik
masuk
memasukkan berita id  865647
Insert berita  Bukber Calya Sigra Club, Berbagi Kebahagiaan Sembari Servis
masuk
memasukkan berita id  859960
Insert berita  Cerita 5 Mobil Toyota Menjelajah Indonesia-Timor Leste Sejauh 8.000 Km
masuk
memasukkan berita id  858782
Insert berita  Kopdar Gabungan Auto2000 with Community Hebohkan Cikarang
masuk
memasukkan berita id  856393
Insert berita  Coba Foto Mobi

[{'subcategory': 'hobi-komunitas',
  'id': 871315,
  'category': 'otomotif',
  'url': 'https://www.seva.id/otomotif/blog/digelar-2-hari-bimmerjunction-2018-lebih-seru-dari-sebelumnya/',
  'pubdate': '2018-07-31 06:57:11',
  'author': 'Ferdi',
  'title': 'Digelar 2 Hari, BimmerJunction 2018 Lebih Seru dari Sebelumnya',
  'source': 'seva',
  'comments': 0,
  'tags': 'acara bimmerjunction 2018,bimmerjunction 2018 bmw,bmw di bimmerjunction',
  'images': 'https://www.seva.id/otomotif/wp-content/uploads/sites/2/2018/08/WhatsApp-Image-2018-07-30-at-4.10.52-PM-1-1024x576.jpeg',
  'content': 'BMW Car Clubs Indonesia (BMWCCI) Jakarta Chapter, kembali mengadakan BimmerJuction 2018 yang diselenggarakan di Q-Big BSD City, Tangerang, Banten. Event ini digelar selama dua hari, yaitu 28 – 29 Juli 2018. Berbeda dengan tahun sebelumnya yang diadakan dalam satu hari.Damar Tias, Ketua Pelaksana BimmerJunction 2018, mengungkapkan, event ini sengaja dibuat dalam dua hari agar pengunjung lebih fleksibel dan 